In [ ]:
# !pip install skorch

# import torchvision.datasets as datasets
# import torchvision.transforms as transforms
# from torchvision.models import resnet50
# from torch import optim
# import torch.nn as nn
# from skorch import NeuralNetClassifier, callbacks
# from skorch.helper import predefined_split
# import os

# def preprocessing(img_path):
#     im = image.load_img(img_path, target_size=(3,224,224))
# #     im = image.img_to_array(im)
# #     im = np.expand_dims(im, axis=0)
#     return im

# model = resnet50(pretrained=True)

# model.fc = nn.Sequential(nn.Linear(2048, 512),
#                          nn.Dropout(0.5),
#                          nn.Linear(512, 2),
#                          nn.Dropout(0.5),
#                          nn.Sigmoid())
# optimizer = optim.SGD
# criterion = nn.CrossEntropyLoss
# net = NeuralNetClassifier(
#     model,
#     optimizer=optimizer,
#     criterion=criterion,
#     device='cuda',
#     max_epochs=10,
#     train_split=None,
#     batch_size=128,
#     iterator_train__shuffle=True,
#     callbacks=[callbacks.ProgressBar(batches_per_epoch='auto'),
#                callbacks.EarlyStopping(monitor='valid_loss', patience=5,
#                                        threshold=1e-5, threshold_mode='rel', lower_is_better=True),
#                callbacks.Checkpoint(monitor='valid_loss_best', f_params='params_best_val.pt', f_optimizer='optimizer_best_val.pt',
#                                     f_criterion='criterion_best_val.pt', f_history='history.json', dirname='./checkpoints', event_name='event_cp')]

# )

# if os.path.isfile('../input/gender-skorch-ipynb/checkpoints/params_best_val.pt'):
#     print('re-initailizing model...')
#     net.initialize()
#     net.load_params(f_params='../input/gender-skorch-ipynb/checkpoints/params_best_val.pt')
# transformer = transforms.Compose([transforms.Resize([128, 128]),
#                                   transforms.ToTensor()])
# def predict_gender(image_path, transformer):
#     img = preprocessing(image_path)
#     img = transformer(img).unsqueeze(0)
#     pred = net.predict_proba(img)
# #     print(pred)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import os
from keras.preprocessing import image, sequence
items = os.listdir('/kaggle/input/medical-masks-part1/images/')
num2words1 = {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', \
            6: 'six', 7: 'seven', 8: 'eight', 9: 'nine', 10: 'ten', \
            11: 'eleven', 12: 'twelve', 13: 'thirteen', 14: 'fourteen', \
            15: 'fifteen', 16: 'sixteen', 17: 'seventeen', 18: 'eighteen', 19: 'nineteen'}
num2words2 = ['twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety']

def number(Number):
    if 0 <= Number <= 19:
        return [num2words1[Number], 1]
    elif 20 <= Number <= 99:
        tens, remainder = divmod(Number, 10)
        if remainder:
            return [[num2words2[tens - 2], num2words1[remainder]], 2]
        else: 
            return [num2words2[tens - 2], 1]
    else:
        return ['zero', 1]
data_cap = []
names = []
captions = []
for name in items:
    token = name.split('_')
    age, lenght = number(int(token[-1].split('.')[0]))
    sex = token[3].lower()
    if sex =='none':
        non_sex = True
    else:
        image_path = os.path.join('/kaggle/input/medical-masks-part1/images/', name)
#         predict_gender(image_path, transformer=transformer)
        non_sex = False
        
    mask_wear_type = number(int(token[1]))[0]
    if lenght == 1:
        token = [name + '#0\t' + age +' '+ sex +' '+  mask_wear_type]
    else:
        token = [name + '#0\t' + age[0] +' '+ age[1] +' '+ sex +' '+ mask_wear_type]
    if name.endswith(".jpg") and not non_sex:
        names.append(name)
        captions.append(token)
        data_cap.append((name, token))
print(names[-10:], len(names), captions[-10:], len(data_cap))
with open('tokens.txt', 'w') as f:
    for item in captions:
        f.write("%s\n" % item[0])

In [ ]:
# Load libraries
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import os
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.utils import np_utils
import random
from keras.preprocessing import image, sequence
import matplotlib.pyplot as plt

In [ ]:
ls

In [ ]:
# Load data
images_dir = os.listdir('/kaggle/input/medical-masks-part1/images/')
x_train = []
for name in images_dir:
    if 'NONE' not in name:
        x_train.append(name)
images_path = '/kaggle/input/medical-masks-part1/images/'
captions_path = './tokens.txt'
captions = open(captions_path, 'r').read().split("\n")

In [ ]:
# Loading captions as values and images as key in dictionary
tokens = {}

for ix in range(len(captions)-1):
    temp = captions[ix].split("#")
    if temp[0] in tokens:
        tokens[temp[0]].append(temp[1][2:])
    else:
        tokens[temp[0]] = [temp[1][2:]]

In [ ]:
# print(tokens)

In [ ]:
# displaying an image and captions given to it
temp = captions[10].split("#")
from IPython.display import Image, display
z = Image(filename=images_path+temp[0])
display(z)

for ix in range(len(tokens[temp[0]])):
    print(tokens[temp[0]][ix])

In [ ]:
train_dataset = open('flickr_8k_train_dataset.txt','wb')
train_dataset.write(b"image_id\tcaptions\n")

In [ ]:
# Populating the above created files for train, test and validation dataset with image ids and captions for each of these images
for img in x_train:
    if img == '':
        continue
    for capt in tokens[img]:
        caption = "<start> "+ capt + " <end>"
        train_dataset.write((img+"\t"+caption+"\n").encode())
        train_dataset.flush()
train_dataset.close()

In [ ]:
# Loading 50 layer Residual Network Model and getting the summary of the model
from IPython.core.display import display, HTML
display(HTML("""<a href="http://ethereon.github.io/netscope/#/gist/db945b393d40bfa26006">ResNet50 Architecture</a>"""))
model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')
model.summary()
# Note: For more details on ResNet50 architecture you can click on hyperlink given below

In [ ]:
# Helper function to process images
def preprocessing(img_path):
    im = image.load_img(img_path, target_size=(224,224,3))
    im = image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im


In [ ]:
train_data = {}
ctr=0
for ix in x_train:
    if ix == "":
        continue
    if ctr >= 10000:
        break
    ctr+=1
    if ctr%1000==0:
        print(ctr)
    path = images_path + ix
    img = preprocessing(path)
    pred = model.predict(img).reshape(2048)
    train_data[ix] = pred

In [ ]:
with open( "train_encoded_images.p", "wb" ) as pickle_f:
    pickle.dump(train_data, pickle_f )  

In [ ]:
# Loading image and its corresponding caption into a dataframe and then storing values from dataframe into 'ds'
pd_dataset = pd.read_csv("flickr_8k_train_dataset.txt", delimiter='\t')
ds = pd_dataset.values
print(ds.shape)

In [ ]:
pd_dataset.head()

In [ ]:
# Storing all the captions from ds into a list
sentences = []
for ix in range(ds.shape[0]):
    sentences.append(ds[ix, 1])
    
print(len(sentences))

In [ ]:
# First 5 captions stored in sentences
sentences[:5]

In [ ]:
# Splitting each captions stored in 'sentences' and storing them in 'words' as list of list
words = [i.split() for i in sentences]

In [ ]:
# Creating a list of all unique words
unique = []
for i in words:
    unique.extend(i)
unique = list(set(unique))

print(len(unique))

vocab_size = len(unique)

In [ ]:
# Vectorization
word_2_indices = {val:index for index, val in enumerate(unique)}
indices_2_word = {index:val for index, val in enumerate(unique)}

In [ ]:
word_2_indices['<pad>'] = 0
word_2_indices['two'] = 23
indices_2_word[0] = '<pad>'
indices_2_word[23] = 'two'

In [ ]:
vocab_size = len(word_2_indices.keys())
print(vocab_size)

In [ ]:
max_len = 0

for i in sentences:
    i = i.split()
    if len(i) > max_len:
        max_len = len(i)

print(max_len)

In [ ]:
padded_sequences, subsequent_words = [], []

for ix in range(ds.shape[0]):
    partial_seqs = []
    next_words = []
    text = ds[ix, 1].split()
    text = [word_2_indices[i] for i in text]
    for i in range(1, len(text)):
        partial_seqs.append(text[:i])
        next_words.append(text[i])
    padded_partial_seqs = sequence.pad_sequences(partial_seqs, max_len, padding='post')

    next_words_1hot = np.zeros([len(next_words), vocab_size], dtype=np.bool)
    
    #Vectorization
    for i,next_word in enumerate(next_words):
        next_words_1hot[i, next_word] = 1
        
    padded_sequences.append(padded_partial_seqs)
    subsequent_words.append(next_words_1hot)
    
padded_sequences = np.asarray(padded_sequences)
subsequent_words = np.asarray(subsequent_words)

print(padded_sequences.shape)
print(subsequent_words.shape)

In [ ]:
print(padded_sequences[0])

In [ ]:
for ix in range(len(padded_sequences[0])):
    for iy in range(max_len):
        print(indices_2_word[padded_sequences[0][ix][iy]],)
    print("\n")

print(len(padded_sequences[0]))

In [ ]:
num_of_images = 2000

In [ ]:
captions = np.zeros([0, max_len])
next_words = np.zeros([0, vocab_size])

In [ ]:
for ix in range(num_of_images):#img_to_padded_seqs.shape[0]):
    captions = np.concatenate([captions, padded_sequences[ix]])
    next_words = np.concatenate([next_words, subsequent_words[ix]])

np.save("captions.npy", captions)
np.save("next_words.npy", next_words)

print(captions.shape)
print(next_words.shape)

In [ ]:
with open('./train_encoded_images.p', 'rb') as f:
    encoded_images = pickle.load(f, encoding="bytes")

In [ ]:
imgs = []
# print(ds.shape[0], ds[0,0], encoded_images[ds[0,0]])
for ix in range(ds.shape[0]):
    if ds[ix, 0] in encoded_images.keys():
#         print(ix, encoded_images[ds[ix, 0]])
        imgs.append(list(encoded_images[ds[ix, 0]]))
        ####### aldaa baigaa
imgs = np.asarray(imgs)
print(imgs.shape)

In [ ]:
images = []

for ix in range(num_of_images):
#     print(padded_sequences[ix].shape)
    for iy in range(padded_sequences[ix].shape[0]):
        images.append(imgs[ix+1])
        
images = np.asarray(images)

np.save("images.npy", images)


In [ ]:
image_names = []

for ix in range(num_of_images):
    for iy in range(padded_sequences[ix].shape[0]):
        image_names.append(ds[ix, 0])
        
image_names = np.asarray(image_names)

np.save("image_names.npy", image_names)

print(len(image_names))

In [ ]:
captions = np.load("captions.npy")
next_words = np.load("next_words.npy")

print(captions.shape)
print(next_words.shape)

In [ ]:
images = np.load("images.npy")

print(images.shape)

In [ ]:
imag = np.load("image_names.npy")
        
print(imag.shape)

In [ ]:
embedding_size = 128
max_len = 6

In [ ]:
image_model = Sequential()

image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))

image_model.summary()

In [ ]:
language_model = Sequential()

language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))

language_model.summary()

In [ ]:
conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs = out)

# model.load_weights("../input/model_weights.h5")
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

In [ ]:
hist = model.fit([images, captions], next_words, batch_size=512, epochs=250)

In [ ]:
model.save_weights("./model_weights.h5")

In [ ]:
def preprocessing(img_path):
    im = image.load_img(img_path, target_size=(224,224,3))
    im = image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im

In [ ]:
def get_encoding(model, img):
    image = preprocessing(img)
    pred = model.predict(image).reshape(2048)
    return pred

In [ ]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')

In [ ]:
img = "../input/medical-masks-part1/images/000015_4_000015_FEMALE_31.jpg"

test_img = get_encoding(resnet, img)

In [ ]:
def predict_captions(image):
    start_word = ["<start>"]
    while True:
        par_caps = [word_2_indices[i] for i in start_word]
        par_caps = sequence.pad_sequences([par_caps], maxlen=max_len, padding='post')
        preds = model.predict([np.array([image]), np.array(par_caps)])
        word_pred = indices_2_word[np.argmax(preds[0])]
        start_word.append(word_pred)
        
        if word_pred == "<end>" or len(start_word) > max_len:
            break
            
    return ' '.join(start_word[1:-1])

Argmax_Search = predict_captions(test_img)

In [ ]:
z = Image(filename=img)
display(z)

print(Argmax_Search)